# Tennis simulation

In [1]:
score_seq = [0, 15, 30, 40, 'A', 'W']

def score_point(player, points):

    if points[player] == 40 and points[player == 0] not in [40, 'A']:
        return ['W' if player == 0 else points[0], 'W' if player == 1 else points[1]]

    if (points[player] == 40) and (points[player == 0] == 'A'):
        return [40, 40]

    a = points.copy()
    a[player] = score_seq[score_seq.index(points[player]) + 1]
    return a


def test(player, given_score, expected_score):
    score = score_point(player, given_score)
    assert score == expected_score, f"Expected {expected_score} but got {score}"

test_cases = [
    [0, [ 0,  0], [15,  0]],
    [0, [15,  0], [30,  0]],
    [0, [30,  0], [40,  0]],
    [0, [40,  0], ['W', 0]],
    [0, ['A', 0], ['W', 0]],
    [0, [40, 40], ['A', 40]],
    [0, ['A', 40], ['W', 40]],
    [0, [40, 'A'], [40, 40]],
    [1, [ 0, 15], [ 0, 30]],
    [1, [ 0, 30], [ 0, 40]],
    [1, [ 0, 40], [ 0, 'W']],
    [1, [ 0, 'A'], [ 0, 'W']],
    [1, [40, 40], [40, 'A']],
    [1, [40, 'A'], [40, 'W']],
    [1, ['A', 40], [40, 40]]
]

for t in test_cases:
    test(*t)

In [8]:
import random

max_matches = 10
win_prob_a = 0.5
games_per_set = 6
sets_per_match = 2

matches_a = matches_b = 0

while matches_a + matches_b < max_matches:
    sets_a = sets_b = 0
    while sets_a < sets_per_match and sets_b < sets_per_match:
        games_a = games_b = 0
        while games_a < games_per_set and games_b < games_per_set:
            points = [0, 0]
            while True:
                print(matches_a, matches_b, sets_a, sets_b, games_a, games_b, points)
                if points[0] == 'W' or points[1] == 'W':
                    break
                points = score_point(random.random() > win_prob_a, points)
            games_a += points[0] == 'W'
            games_b += points[1] == 'W'
        sets_a += games_a > games_b
        sets_b += games_b > games_a
    matches_a += sets_a > sets_b
    matches_b += sets_b > sets_a

0 0 0 0 0 0 [0, 0]
0 0 0 0 0 0 [15, 0]
0 0 0 0 0 0 [30, 0]
0 0 0 0 0 0 [30, 15]
0 0 0 0 0 0 [40, 15]
0 0 0 0 0 0 [40, 30]
0 0 0 0 0 0 [40, 40]
0 0 0 0 0 0 ['A', 40]
0 0 0 0 0 0 [40, 40]
0 0 0 0 0 0 [40, 'A']
0 0 0 0 0 0 [40, 'W']
0 0 0 0 0 1 [0, 0]
0 0 0 0 0 1 [15, 0]
0 0 0 0 0 1 [30, 0]
0 0 0 0 0 1 [30, 15]
0 0 0 0 0 1 [30, 30]
0 0 0 0 0 1 [40, 30]
0 0 0 0 0 1 [40, 40]
0 0 0 0 0 1 [40, 'A']
0 0 0 0 0 1 [40, 'W']
0 0 0 0 0 2 [0, 0]
0 0 0 0 0 2 [15, 0]
0 0 0 0 0 2 [30, 0]
0 0 0 0 0 2 [40, 0]
0 0 0 0 0 2 [40, 15]
0 0 0 0 0 2 ['W', 15]
0 0 0 0 1 2 [0, 0]
0 0 0 0 1 2 [0, 15]
0 0 0 0 1 2 [15, 15]
0 0 0 0 1 2 [15, 30]
0 0 0 0 1 2 [15, 40]
0 0 0 0 1 2 [15, 'W']
0 0 0 0 1 3 [0, 0]
0 0 0 0 1 3 [0, 15]
0 0 0 0 1 3 [0, 30]
0 0 0 0 1 3 [15, 30]
0 0 0 0 1 3 [15, 40]
0 0 0 0 1 3 [30, 40]
0 0 0 0 1 3 [40, 40]
0 0 0 0 1 3 [40, 'A']
0 0 0 0 1 3 [40, 'W']
0 0 0 0 1 4 [0, 0]
0 0 0 0 1 4 [15, 0]
0 0 0 0 1 4 [30, 0]
0 0 0 0 1 4 [30, 15]
0 0 0 0 1 4 [40, 15]
0 0 0 0 1 4 ['W', 15]
0 0 0 0 2 4 [0, 0]
0 0 0 0 

In [23]:
import pandas as pd
from io import StringIO

# Avoids SettingWithCopyWarning
pd.options.mode.copy_on_write = True

data_io = StringIO(cap.stdout)
df = pd.read_csv(data_io, sep='\t')